In [187]:
from python_storybook.core import Story, StoryManager, StoryHub

In [230]:
from typing import List, Any, Dict, Literal, Optional, Union, get_type_hints

def with_typehint(arg1:str, arg2:Optional[List[str]], arg3:Literal["A", "B"], arg4:Union[str, Literal["A", "B"]])->str:
    return {"string":[1, "another string"]}

In [266]:
story_manager = StoryManager(title="Example")

story_manager.create_story(
    func=with_typehint,
    kwargs={
        "arg1":"Hello world",
        "arg2":["hi", "buy"],
        "arg3":"A",
        "arg4":'I am boring.'
    }
)

In [268]:
story_meta = StoryHub.get_story_meta(story_manager.get_stories()[0])

In [269]:
story = story_manager.get_stories()[0]

In [270]:
get_type_hints(with_typehint)

{'arg1': str,
 'arg2': typing.Optional[typing.List[str]],
 'arg3': typing.Literal['A', 'B'],
 'arg4': typing.Union[str, typing.Literal['A', 'B']],
 'return': str}

In [258]:
type(get_type_hints(with_typehint)['arg4'])

typing._UnionGenericAlias

In [233]:
type(get_type_hints(with_typehint)['return'])

type

In [192]:
type_hints = story_meta.type_hints
type_hints

{'arg1': 'str',
 'arg2': 'Optional[List[str]]',
 'arg3': "Literal['A', 'B']",
 'arg4': "Union[str, Literal['A', 'B']]",
 'return': 'str'}

In [193]:
def get_indexes_by_bracket(text:str, target:str, open='[', close=']', reversed=True):
    if reversed:
        start_index = text.rfind(target)
    else:
        start_index = text.find(target)
    bracket_start_index = text[start_index:].find(open) + start_index
    end_index = bracket_start_index
    count = 0
    for letter in text[bracket_start_index:]:
        if letter==open:
            count += 1
        elif letter==close:
            count -= 1
        else:
            pass
        end_index += 1
        if count == 0:
            break
    return start_index, end_index

In [194]:
start_index, end_index = get_indexes_by_bracket('Optional[List[List[]]]', "List")

In [195]:
'Optional[List[List[]]]'[start_index:end_index]

'List[]'

In [196]:
def count_repetation(text, target):
    split = text.split(target)
    return len(split) - 1

In [197]:
def handle_Literal(text:str):
    split = text.replace("Literal", "")[1:-1].split(',')
    text = " | ".join(split)
    return text

In [198]:
def handle_Union(text:str):
    split = text.replace("Union", "")[1:-1].split(',')
    text = " | ".join(split)
    return text

In [199]:
handle_Literal("Literal[str, float]")

'str |  float'

In [200]:
def handle_List(text:str):
    text = text.replace("List", "")[1:-1] + "[]"
    return text

In [201]:
handle_List("List[str]")

'str[]'

In [202]:
def handle_Optional(text:str):
    text = text.replace("Optional", "")[1:-1] + "?"
    return text

In [203]:
'Dict[str, Any]'

'Dict[str, Any]'

In [204]:
def handle_Dict(text:str):
    template = "[key: {key_type}]: {value_type}"
    key_type, value_type = text.replace("Dict", "")[1:-1].split(",")
    text = template.format(key_type=key_type, value_type=value_type)
    text = "{ " + text + " }"
    return text
    

In [205]:
handle_Dict('Dict[str, Any]')

'{ [key: str]:  Any }'

In [206]:

def handle_base(text:str):
    text = text.replace("str", "string")
    text = text.replace("float", "number")
    text = text.replace("int", "number")
    text = text.replace("Any", "any")
    return text



In [207]:
handlers = {
    "base":handle_base,
    "List":handle_List,
    "Dict":handle_Dict,
    "Literal":handle_Literal,
    "Optional":handle_Optional,
    "Union":handle_Union,
}

special_types = list(handlers.keys())
special_types.remove("base")

In [208]:
special_types

['List', 'Dict', 'Literal', 'Optional', 'Union']

In [209]:
def converting(text, target, handlers=handlers):
    start_index, end_index = get_indexes_by_bracket(text, target)
    split = text[:start_index], text[start_index:end_index], text[end_index:]
    for i in range(len(split)):
        if split[i] is None:
            split[i] = ""
    front, middle, back = split
    middle =  handlers[target](middle)
    text = front + middle + back
    return text

In [210]:
def get_last_special_type(text:str, special_types:List[str])->Union[str, None]:
    last_special = None
    last_index = -1
    
    for special_type in special_types:
        current_index = text.rfind(special_type)
        if current_index > last_index:
            last_index = current_index
            last_special = special_type

    return last_special

In [211]:
get_last_special_type('Optional[List[List[str]]]', special_types=special_types)

'List'

In [212]:
def map_python_to_typescript(text:str, special_types=special_types, handlers=handlers, buffer=20):
    for _ in range(buffer):
        special_type = get_last_special_type(text, special_types)
        if special_type is not None:
            text = converting(text, special_type, handlers)
        else:
            break
    text = handle_base(text)
    return text

In [213]:
map_python_to_typescript('Optional[List[List[str]]]')

'string[][]?'

In [214]:
converting('Optional[List[List[str]]]', 'List', handlers)

'Optional[List[str[]]]'

In [215]:
converted = converting(converting('Optional[List[List[str]]]', 'List'), 'List')
converted

'Optional[str[][]]'

In [216]:
map_python_to_typescript('List[str]')

'string[]'

In [217]:
boolen = None

if 'List[str]'.find("List")==-1:
    boolen = True
else:
    boolen = False
boolen

False

In [218]:
type_hints

{'arg1': 'str',
 'arg2': 'Optional[List[str]]',
 'arg3': "Literal['A', 'B']",
 'arg4': "Union[str, Literal['A', 'B']]",
 'return': 'str'}

In [219]:
Optional[Optional[str]]

typing.Optional[str]

In [ ]:
return_prop = converted.pop( 'return')

In [220]:

def post_Optional(key, value):
    if value[-1] == "?":
        key = key + "?"
        value = value[:-1]
    return key, value

post_processors = [post_Optional]


In [221]:
def convert_python_to_typescript(type_hints:Dict[str, str], post_processors=post_processors)->Dict[str, str]:
    converted = {}
    for key, value in type_hints.items():
        value = map_python_to_typescript(value)
        for post_processor in post_processors:
            key, value = post_processor(key, value)
        converted[key] = value
    return converted

In [222]:
converted = convert_python_to_typescript(type_hints=type_hints)
converted

{'arg1': 'string',
 'arg2?': 'string[]',
 'arg3': "'A' |  'B'",
 'arg4': "string |  'A' |  'B'",
 'return': 'string'}

In [223]:
converted

{'arg1': 'string',
 'arg2?': 'string[]',
 'arg3': "'A' |  'B'",
 'arg4': "string |  'A' |  'B'",
 'return': 'string'}

True

'string'

In [238]:
converted

{'arg1': 'string',
 'arg2?': 'string[]',
 'arg3': "'A' |  'B'",
 'arg4': "string |  'A' |  'B'"}

In [224]:
def convert_dict_to_string(dict):
    text = "{\n"
    for key, value in dict.items():
        text = text + f"\t{key}: {value}; \n"
    text = text + "}"
    return text

In [252]:


is_return_prop_type = isinstance(type, type(get_type_hints(with_typehint)['return']))

return_interface = ""

if is_return_prop_type is False:
    return_interface = convert_dict_to_string(return_prop)

return_interface

''

In [240]:
print(convert_dict_to_string(converted))

{
	arg1: string; 
	arg2?: string[]; 
	arg3: 'A' |  'B'; 
	arg4: string |  'A' |  'B'; 
}


In [226]:
def convert(func_name, type_hints):
    typescript_form = convert_python_to_typescript(type_hints)
    string_form = convert_dict_to_string(typescript_form)
    string_form = f"interface {func_name}Props " + string_form
    return string_form

In [227]:
print(convert("MyFunction", type_hints=type_hints))

interface MyFunctionProps {
	arg1: string; 
	arg2?: string[]; 
	arg3: 'A' |  'B'; 
	arg4: string |  'A' |  'B'; 
	return: string; 
}


In [271]:
story.kwargs

{'arg1': 'Hello world',
 'arg2': ['hi', 'buy'],
 'arg3': 'A',
 'arg4': 'I am boring.'}

In [316]:
def convert_dict_to_string2(dict):
    text = "{\n"
    for key, value in dict.items():
        if isinstance(value, str):
            value = '"' + value + '"'
        text = text + f"\t{key}: {value}, \n"
    text = text + "}"
    return text

In [317]:
converted_kwarg = convert_dict_to_string2(story.kwargs)

In [318]:
converted_kwarg

'{\n\targ1: "Hello world", \n\targ2: [\'hi\', \'buy\'], \n\targ3: "A", \n\targ4: "I am boring.", \n}'

In [301]:
print(converted_kwarg)

{
	arg1: "Hello world"; 
	arg2: ['hi', 'buy']; 
	arg3: "A"; 
	arg4: "I am boring."; 
}


In [319]:
def generate_function_string(function_name:str, kwargs:Dict[str,Any], return_prop):
    converted_kwarg = convert_dict_to_string2(kwargs)
    template = 'const {function_name} = ( inputs: {function_name}Props = {converted_kwarg}): {return_prop} => '
    formatted = template.format(function_name=function_name, converted_kwarg=converted_kwarg, return_prop=return_prop)
    return formatted + r'{ return "hi" }'

In [320]:
print(generate_function_string("MyFunction", story.kwargs, "string"))

const MyFunction = ( inputs: MyFunctionProps = {
	arg1: "Hello world", 
	arg2: ['hi', 'buy'], 
	arg3: "A", 
	arg4: "I am boring.", 
}): string => { return "hi" }


interface MyFunctionReturn {
    [key: string]:  any 
}

const MyFunction2 = (
    inputs: MyFunctionProps2 = {
        arg1: 'defaultString',
        arg2: ['default1', 'default2'],
        arg3: 'A',
        arg4: 'B'
    }
): string => {
    return "{ hi: 1 }";
};


In [228]:
story.kwargs

{'arg1': 'Hello world', 'arg3': 'A'}